![net4](net4.png)
* 当f1点发生单相接地短路时，求变压器T1三角侧的各相电流和电压。
* 当f1点单相接地短路时，计算短路点f1的各序电流、电压分量，流过线路l1中的I2 、I0及 l1中的各相电流， 
* 试计算在f2点发生两相短路接地时流过线路l1及l2中的零序电流及l2线路首末端的零序电压。

In [1]:
import numpy as np
import scipy as sp
import ppsa as psa

In [ ]:
def short_circuit1(self,f,loc,Ii):
    Z = self.impedance_matrix()
    E=np.zeros((self.node_num,len(loc)),dtype=np.complex128)
    loc = np.array(loc) - 1
    E[loc,range(len(loc))]=Ii
    Ue = Z.dot(E)
    I = Ue[f-1]/Z[f-1,f-1]
    return I

In [ ]:
def transf3(Vs1_2,Vs2_3,Vs3_1):
    return np.dot([[0.5,-0.5,0.5],
                   [0.5,0.5,-0.5],
                   [-0.5,0.5,0.5]],[Vs1_2,Vs2_3,Vs3_1])

In [12]:
def part2total(a1,a2,a0):
    a = np.e**(np.pi*2/3)
    abc = np.dot([[1 ,1, 1],
                 [a**2,a,1],
                 [a,a**2,1]],[a1,a2,a0])
    return abc

In [ ]:
def current_distribution(admat,U):
    U= np.array(U)
    U= U.reshape((-1,1))-U.reshape((1,-1))
    I = -U * admat
    return I

In [2]:
Sb = 31.5
gd4_1 = psa.power_grid(5,Sb)
gd4_2 = psa.power_grid(5,Sb)
gd4_0 = psa.power_grid(3,Sb)

Xt2_1,Xt2_2,Xt2_3 = 0.1075,0.0625,0

# 正序网络
gd4_1.add_load((1,0),800,0.6j,ref_vol=1j)
gd4_1.add_line((1,3),115,0.4,100)
gd4_1.add_line((1,2),115,0.4,30)
gd4_1.add_line((2,3),115,0.4,60)
gd4_1.add_transformer((3,4),31.5,0.105)
gd4_1.add_transformer((3,5),31.5,Xt2_1)
gd4_1.add_generator((4,0),25/0.8,0.1138j,ref_vol=1j)
gd4_1.add_generator((5,0),25/0.8,0.1138j,ref_vol=1j)

# 负序网络
gd4_2.add_load((1,0),800,0.6j)
gd4_2.add_line((1,3),115,0.4,100)
gd4_2.add_line((1,2),115,0.4,30)
gd4_2.add_line((2,3),115,0.4,60)
gd4_2.add_transformer((3,4),31.5,0.105)
gd4_2.add_transformer((3,5),31.5,Xt2_1)
gd4_2.add_generator((4,0),25/0.8,0.1138j)
gd4_2.add_generator((5,0),25/0.8,0.1138j)

# 零序网络
gd4_0.add_load((1,0),800,0.6j)
gd4_0.add_line((1,3),115,0.4,100)
gd4_0.add_line((1,2),115,0.4*3,30)
gd4_0.add_line((2,3),115,0.4*3,60)
gd4_0.add_transformer((2,0),20,0.105)
gd4_0.add_transformer((3,0),31.5,Xt2_1)

In [8]:
Ub =115
Ib = 31.5/(np.sqrt(3)*Ub)

In [9]:
f = 3    #短路点
Z1 = gd4_1.Z;Z2 = gd4_2.Z;Z0 = gd4_0.Z    #各序阻抗矩阵
U = gd4_1.nodes_voltage()
Ifa1 = U[f-1]/(Z1[f-1,f-1]+Z2[f-1,f-1]+Z0[f-1,f-1])    #a相正序电流
Ifa = 3*Ifa1    #a相电流总合
Ufa1= U[f-1] - Z1[f-1,f-1]*Ifa1    #短路后3结点正序电压
Ufa2= -Z2[f-1,f-1]*Ifa1
Ufa0= -Z0[f-1,f-1]*Ifa1

print('f1短路后a相正、负、零序电流：',Ifa1*Ib,'\n','正、负、零序电压：',Ufa1*Ub,'\t',Ufa2*Ub,'\t',Ufa0*Ub)

f1短路后a相正、负、零序电流： (0.377165622884+0j) 
 正、负、零序电压： 78.099965482j 	 -36.900034518j 	 -41.1999309641j


In [13]:
Xt1 = psa.impedance_of_transformer(0.105,Sb,31.5)
Xg1 = psa.impedance_of_generator(0.1138,Sb,25/0.8)
Xl1_2=psa.impedance_of_line(50,0.4,115,Sb)
Xl1_1=Xl1_2
Xl1_0=psa.impedance_of_line(50,0.4*3,115,Sb)

In [10]:
It1_1 = (1j - Ufa1)/(Xt1*1j+Xg1*1j)
It1_2 = -Ufa2/(Xt1*1j+Xg1*1j)
It1 = It1_1+It1_2
print('变压器T1三角侧的短路电流',It1*Ib)

变压器T1三角侧的短路电流 (0.145502666672+0j)


In [23]:
Il1_2 = (Ufa2-0)/Xl1_2    #流过L1的负序电流
Il1_0 = (Ufa0-0)/Xl1_0
Il1_1 = (Ufa1-U[0])/Xl1_1
Il_abc=part2total(Il1_1,Il1_2,Il1_0)
print('流过L1的负序电流:',Il1_2*Ib,'零序电流:',Il1_0*Ib)
print('流过L1的各相电流:',Il_abc*Ib)

流过L1的负序电流: -0.335541856577j 零序电流: -0.124880653972j
流过L1的各相电流: [ 0. -0.79596437j  0.-24.97628246j  0.-24.97628246j]
